<a href="https://colab.research.google.com/github/adrijr009/leaves-classification/blob/main/leaves_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VC - Classificação de Folhas

## Importar o Dataset pela API

###Fazer o import do .zip pelo Kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauravneupane/flavia-dataset")

print("Path to dataset files:", path)

###Printar as imagens ( Só pra testar)

In [ ]:
import os
import math
import matplotlib.pyplot as plt
from skimage.io import imread

folder = "/root/.cache/kagglehub/datasets/gauravneupane/flavia-dataset/versions/1/Leaves"
all_files = sorted(os.listdir(folder))[:1907] # Seus 1907 arquivos

# Configuração
batch_size = 20 # Imagens por figura (4x5)
total_batches = math.ceil(len(all_files) / batch_size)

print(f"Gerando {total_batches} grupos de imagens. Isso pode demorar um pouco...")

# Loop pelos lotes (chunks)
for batch_idx in range(total_batches):
    start = batch_idx * batch_size
    end = start + batch_size
    batch_files = all_files[start:end]

    # Cria uma NOVA figura para cada lote de 20
    plt.figure(figsize=(15, 12))
    plt.suptitle(f"Lote {batch_idx + 1} de {total_batches}", fontsize=16)

    for i, fname in enumerate(batch_files):
        img_path = os.path.join(folder, fname)
        try:
            img = imread(img_path)

            # Subplot reinicia a contagem para cada figura (1 a 20)
            plt.subplot(4, 5, i+1)
            plt.imshow(img)
            plt.title(fname, fontsize=8)
            plt.axis("off")
        except:
            pass

    plt.tight_layout()
    plt.show() # Mostra este lote e libera memória para o próximo

    # Dica: Se quiser parar no meio para não gerar centenas de imagens,
    # descomente as linhas abaixo:
    # if batch_idx == 2:
    #     print("Parando demonstração para economizar tempo.")
    #     break

## Pré-processamento, Segmentação e Extração de Descritores

### Import das imagens

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# Caminho do dataset
path = "/root/.cache/kagglehub/datasets/gauravneupane/flavia-dataset/versions/1/Leaves"

### Função de segmentação da imagem

In [ ]:
def processar_folha(caminho_img, nome_arquivo):
    # 1. Leitura e Conversão para Tons de Cinza
    img = cv2.imread(caminho_img)
    if img is None:
        return None

    # Conversão direta BGR -> Gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. Limiarização Automática (Otsu)
    # Aplicamos um leve GaussianBlur antes apenas para reduzir ruído de sensor,
    # ajudando o Otsu a ser mais estável sem perder a forma.
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # THRESH_BINARY_INV + THRESH_OTSU:
    # O OpenCV calcula o limiar ideal automaticamente.
    # Usamos INV assumindo fundo claro e folha escura.
    val_otsu, binaria = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # 3. Limpeza e Eliminação de Artefatos (Morfologia)
    # Operação de "Fechamento" (Closing) para fechar pequenos buracos DENTRO da folha
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    binaria = cv2.morphologyEx(binaria, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Operação de "Abertura" (Opening) para remover ruídos NO FUNDO (fora da folha)
    binaria = cv2.morphologyEx(binaria, cv2.MORPH_OPEN, kernel, iterations=2)

    # 4. Extração do Contorno Principal
    # RETR_EXTERNAL: Pega apenas os contornos externos (ignora buracos internos da folha para cálculo de forma global)
    # CHAIN_APPROX_NONE: Guarda TODOS os pontos do contorno (máxima precisão).
    # Se quiser economizar memória sem perder muita precisão, use CHAIN_APPROX_SIMPLE.
    contornos, _ = cv2.findContours(binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if not contornos:
        return None

    # Seleciona o maior contorno pela área (a folha)
    c_folha = max(contornos, key=cv2.contourArea)
    area = cv2.contourArea(c_folha)

    # Filtro de segurança: ignora se for apenas um ruído que sobrou
    if area < 1000:
        return None

    # --- Extração de Descritores (Conforme Enunciado) ---

    # A) Circularidade / Compacidade: C = (4 * pi * A) / P^2
    perimetro = cv2.arcLength(c_folha, True) # True indica que o contorno é fechado
    if perimetro == 0:
        return None

    circularidade = (4 * np.pi * area) / (perimetro ** 2)

    # B) Excentricidade
    # Ajusta uma elipse ao redor do contorno e compara o eixo maior com o menor
    if len(c_folha) >= 5: # Necessário pelo menos 5 pontos para fitEllipse
        (x, y), (eixo_menor, eixo_maior), angle = cv2.fitEllipse(c_folha)

        # Garante que a divisão seja sempre do menor pelo maior
        # Nota: fitEllipse pode retornar largura/altura em ordem variada dependendo da rotação
        a = max(eixo_menor, eixo_maior) / 2.0
        b = min(eixo_menor, eixo_maior) / 2.0

        # Fórmula: e = sqrt(1 - (b^2 / a^2))
        # Se for um círculo perfeito, b=a, excentricidade = 0. Linha reta = 1.
        excentricidade = np.sqrt(1 - (b**2 / a**2))
    else:
        excentricidade = 0.0

    # C) Razão Altura / Largura (Bounding Box Retangular)
    x, y, w, h = cv2.boundingRect(c_folha)
    razao_hw = float(h) / w if w != 0 else 0

    # D) Número de Cantos (Shi-Tomasi)
    # Shi-Tomasi (goodFeaturesToTrack) é geralmente mais robusto para *contagem* que o Harris puro
    # Máscara da folha para não pegar cantos do fundo
    mask_folha = np.zeros_like(gray)
    cv2.drawContours(mask_folha, [c_folha], -1, 255, -1)

    # Parâmetros: maxCorners=0 (ilimitado), qualityLevel=0.01, minDistance=10 pixels
    # minDistance evita detectar vários cantos no mesmo "bico" da folha
    cantos = cv2.goodFeaturesToTrack(gray, maxCorners=0, qualityLevel=0.01, minDistance=10, mask=mask_folha)

    if cantos is not None:
        num_cantos = len(cantos)
    else:
        num_cantos = 0

    return {
        "Imagem": nome_arquivo,
        "Area": area,
        "Perimetro": perimetro,
        "Circularidade": circularidade,
        "Excentricidade": excentricidade,
        "Num_Cantos": num_cantos,
        "Razao_Altura_Largura": razao_hw
    }

### Processando todo dataset

In [ ]:
# --- Execução Serial (Mais segura para RAM com imagens Full-Res) ---
# o loop simples é mais garantido de não estourar a memória.

imagens = sorted([
    f for f in os.listdir(path)
    if f.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif'))
])

registros = []

print(f"Iniciando processamento de alta precisão em {len(imagens)} imagens...")

for nome in tqdm(imagens):
    caminho = os.path.join(path, nome)
    try:
        dados = processar_folha(caminho, nome)
        if dados:
            registros.append(dados)
    except Exception as e:
        print(f"Erro ao processar {nome}: {e}")

### Criação e Salvamento dos Descritores

In [ ]:
df = pd.DataFrame(registros)

print("\nProcessamento Concluído.")
print(df.head())

# Salvar CSV
df.to_csv("descritores_folhas.csv", index=False, sep=';')

## Redução de Dimensionalidade (PCA)

### Leitura do CSV

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [ ]:
# Carregar o CSV gerado anteriormente
df = pd.read_csv("descritores_folhas.csv", sep=';')

print("Formato do DataFrame:", df.shape)
display(df.head())


### Normalização dos Descritores

In [ ]:
# Selecionar SOMENTE os descritores numéricos
X = df[[
    "Circularidade",
    "Excentricidade",
    "Num_Cantos",
    "Razao_Altura_Largura"
]].values

# Normalização (z-score)
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

print("Média após normalização:", X_norm.mean(axis=0))
print("Desvio padrão após normalização:", X_norm.std(axis=0))


### Aplicação PCA

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X_norm)

# Variância explicada
var_exp = pca.explained_variance_ratio_
var_exp_acumulada = np.cumsum(var_exp)

for i, v in enumerate(var_exp_acumulada):
    print(f"Componente {i+1}: Variância acumulada = {v:.4f}")


### Justificativa

As duas primeiras componentes principais explicam aproximadamente 79,8% da variância total, sendo suficientes para análise exploratória e visualização em 2D, enquanto três componentes explicam 92,8% da variância, podendo ser utilizadas quando se deseja maior fidelidade na representação dos dados.

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(var_exp_acumulada)+1),
         var_exp_acumulada,
         marker='o')

plt.xlabel("Número de Componentes")
plt.ylabel("Variância Explicada Acumulada")
plt.title("Análise da Variância Explicada (PCA)")
plt.grid(True)
plt.show()


### PCA 2 Componentes

In [ ]:
# Escolha livre dos componentes (começa em 1 para ficar intuitivo)
comp_x = 1   # PC1
comp_y = 2   # PC2

plt.figure(figsize=(8,6))
plt.scatter(
    X_pca[:, comp_x-1],
    X_pca[:, comp_y-1],
    s=25
)

plt.xlabel(f"PC{comp_x}")
plt.ylabel(f"PC{comp_y}")
plt.title(f"PCA: PC{comp_x} × PC{comp_y}")
plt.grid(True)
plt.show()


## Classificação

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
import cv2
import os

# Importações de Machine Learning
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

### Preparação dos Dados

In [ ]:
# --- CONFIGURAÇÃO ---
path = "/root/.cache/kagglehub/datasets/gauravneupane/flavia-dataset/versions/1/Leaves"

# --- 1. Mapeamento das Classes Flavia ---
flavia_intervals = {
    1: (1001, 1059),  2: (1060, 1122),  3: (1123, 1194),  4: (1195, 1267),
    5: (1268, 1323),  6: (1324, 1385),  7: (1386, 1437),  8: (1438, 1496),
    9: (1497, 1551), 10: (1552, 1616), 11: (2001, 2050), 12: (2051, 2113),
    13: (2114, 2165), 14: (2166, 2230), 15: (2231, 2290), 16: (2291, 2346),
    17: (2347, 2423), 18: (2424, 2485), 19: (2486, 2546), 20: (2547, 2612),
    21: (2616, 2675), 22: (3001, 3055), 23: (3056, 3110), 24: (3111, 3175),
    25: (3176, 3229), 26: (3230, 3281), 27: (3282, 3334), 28: (3335, 3389),
    29: (3390, 3446), 30: (3447, 3510), 31: (3511, 3563), 32: (3566, 3621)
}

def obter_classe_flavia(nome_imagem):
    try:
        # Remove a extensão e converte para inteiro
        numero = int(nome_imagem.split('.')[0])

        # Verifica em qual intervalo o número se encaixa
        for classe_id, (inicio, fim) in flavia_intervals.items():
            if inicio <= numero <= fim:
                return classe_id

        return -1 # Não encontrado / Outro padrão
    except:
        return -1

# Aplica a correção
df['Classe'] = df['Imagem'].apply(obter_classe_flavia)

# Removemos imagens que porventura não estejam nos intervalos (-1)
df_limpo = df[df['Classe'] != -1].copy()

# Atualiza X e y com o dataframe limpo
colunas_numericas = ["Circularidade", "Excentricidade", "Num_Cantos", "Razao_Altura_Largura"]
X = df_limpo[colunas_numericas].values
y = df_limpo['Classe'].values

# --- 2. Estatísticas e Visualização Corrigida ---
contagem_classes = df_limpo['Classe'].value_counts().sort_index()
classes_unicas = sorted(df_limpo['Classe'].unique())

print("="*40)
print("ESTATÍSTICAS CORRIGIDAS (REAL FLAVIA)")
print("="*40)
print(f"Total de Imagens Válidas: {len(df_limpo)}")
print(f"Total de Classes: {len(classes_unicas)}")
print("-" * 40)
print(f"{'Classe':<10} | {'Qtd Imagens':<15}")
print("-" * 40)
# Mostra apenas as 10 primeiras para não poluir, ou tire o [:10] para ver tudo
for classe in classes_unicas[:10]:
    print(f"{str(classe):<10} | {contagem_classes[classe]:<15}")
print("... (restante das classes omitido)")
print("="*40)

# --- 3. Plotagem das Amostras ---
n_classes = len(classes_unicas)
cols = 5
rows = math.ceil(n_classes / cols)

plt.figure(figsize=(15, 3.5 * rows))
plt.suptitle("Amostra Real: Uma folha de cada espécie Flavia", fontsize=16, y=1.01)

for i, classe in enumerate(classes_unicas):
    # Pega a primeira imagem desta classe
    amostra = df_limpo[df_limpo['Classe'] == classe].iloc[0]
    nome_img = amostra['Imagem']
    caminho_completo = os.path.join(path, nome_img)

    plt.subplot(rows, cols, i + 1)

    try:
        img = cv2.imread(caminho_completo)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.imshow(img)
            plt.title(f"Classe {classe}\n({nome_img})", fontsize=10, backgroundcolor='#eeeeee')
        else:
            plt.text(0.5, 0.5, "Img missing", ha='center')
    except:
        pass

    plt.axis('off')

plt.tight_layout()
plt.show()

### Classificador K-NN e Curva de Desempenho

In [ ]:
# --- 2. Classificador k-NN (Curva de Desempenho) ---

print("\n--- A iniciar testes com k-NN ---")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

k_range = range(1, 21)
k_scores = []

for k in k_range:
    # Pipeline: Normaliza -> Classifica
    # É importante normalizar DENTRO do loop para evitar data leakage na validação cruzada
    knn = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=k))

    # Executa validação cruzada e guarda a média das acurácias
    scores = cross_val_score(knn, X, y, cv=cv, scoring='accuracy')
    k_scores.append(scores.mean())

# Gráfico Desempenho x k
plt.figure(figsize=(10, 6))
plt.plot(k_range, k_scores, marker='o', linestyle='-', color='b')
plt.title('Desempenho k-NN: Acurácia vs Valor de k')
plt.xlabel('Valor de k (Vizinhos)')
plt.ylabel('Acurácia Média (Cross-Validation)')
plt.xticks(k_range)
plt.grid(True)
plt.show()

melhor_k = k_range[np.argmax(k_scores)]
print(f"Melhor k encontrado: {melhor_k} com acurácia de {max(k_scores):.4f}")

### Classificador SVM

In [ ]:
# --- 3. Classificador SVM (Comparação de Kernels) ---

print("\n--- A iniciar testes com SVM ---")

kernels = ['linear', 'rbf']
svm_results = {}

for kernel in kernels:
    # Pipeline: Normaliza -> SVM
    svm = make_pipeline(StandardScaler(), SVC(kernel=kernel, random_state=42))

    scores = cross_val_score(svm, X, y, cv=cv, scoring='accuracy')
    svm_results[kernel] = scores.mean()
    print(f"Kernel '{kernel}': Acurácia média = {scores.mean():.4f}")

# Gráfico comparativo SVM
plt.figure(figsize=(6, 5))
barras = plt.bar(svm_results.keys(), svm_results.values(), color=['skyblue', 'salmon'])
plt.ylim(0, 1.1)
plt.ylabel('Acurácia Média')
plt.title('Comparação de Kernels SVM')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adicionar valores nas barras
for bar in barras:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.02, f"{yval:.2f}", ha='center', va='bottom')

plt.show()


## Avaliação do Sistema

### Divisão Treino e Teste

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Divisão Treino e Teste
# Reservamos 30% dos dados para teste final (dados que o modelo nunca viu)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

### Treinamento do Modelo Escolhido

In [ ]:
# 2. Treinamento do Modelo Escolhido
# Usaremos o SVM com kernel RBF (geralmente o melhor para este caso)
# Se o seu k-NN foi muito melhor, troque SVC por KNeighborsClassifier
model = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, gamma='scale'))
model.fit(X_train, y_train)

### Predição

In [ ]:
# 3. Predição
y_pred = model.predict(X_test)

### Métricas Numéricas (Precisão, Recall, F1, Acurácia)

In [ ]:
# --- A) Métricas Numéricas (Precisão, Recall, F1, Acurácia) ---
print("="*60)
print("RELATÓRIO DE CLASSIFICAÇÃO")
print("="*60)

# O output_dict=False gera o texto formatado.
# O zero_division=0 evita erros se alguma classe não for predita.
report = classification_report(y_test, y_pred, zero_division=0)
acc = accuracy_score(y_test, y_pred)

print(report)
print(f"\nACURÁCIA GLOBAL: {acc:.4f} ({acc*100:.2f}%)")
print("="*60)

### Matriz de Confusão Visual

In [ ]:
# --- B) Matriz de Confusão Visual ---
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(12, 10)) # Tamanho grande para ver todas as classes
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=np.unique(y),
            yticklabels=np.unique(y))

plt.title('Matriz de Confusão (SVM RBF)')
plt.ylabel('Classe Real (Verdade)')
plt.xlabel('Classe Predita (Modelo)')
plt.show()

### Discussão para o Relatório

Os principais erros ocorrem entre espécies que possuem razões de aspecto e circularidades similares (ex: folhas lanceoladas de diferentes famílias). O número de cantos ajudou a diferenciar folhas lisas de folhas serrilhadas, mas a similaridade geométrica pura é um limitador para modelos que não utilizam textura ou cor.
